In [ ]:
import matplotlib.pyplot as pp
import numpy as np
import pandas as pd
import pandas_datareader.data as datareader
import tensorflow as tf
import tensorflow_probability as tp

In [ ]:
%config InlineBackend.figure_format = 'svg'

In [ ]:
pp.style.use('ggplot')

In [ ]:
session = tf.Session()

prior_params_ = {
    'GOOG': (-0.03, 0.04), 
    'AAPL': (0.05, 0.03), 
    'AMZN': (0.03, 0.02), 
    'TSLA': (-0.02, 0.01),
}

In [ ]:
pp.figure(figsize=(12, 6))
for i, (name, params) in enumerate(prior_params_.items()):
    x = tf.linspace(start=-0.15, stop=0.15, num=100)
    pp.subplot(2, 2, i + 1)
    y = tp.distributions.Normal(loc=params[0], scale=params[1]).prob(x)
    [x_, y_] = session.run([x, y])
    pp.fill_between(x_, 0, y_, linewidth=2, alpha=0.6)
    pp.title(name)
    pp.vlines(0, 0, y_.max(), 'k', '--', linewidth=0.5)
    pp.xlim(-0.15, 0.15)
pp.tight_layout()

In [ ]:
stocks = prior_params_.keys()

observation_num = 100
start_date, end_date = '2015-09-01', '2018-12-21'

data = pd.DataFrame()
for name in stocks:
    series = datareader.DataReader(name, 'iex', start_date, end_date)['close']
    data[name] = series

data = data[::-1]
data = data.pct_change()[1:observation_num + 1]
data_ = data.values.astype(dtype=np.float32)